# Euler 004

Euler 004 asks to find the largest palindromic number that can be created from
the product of two `4` digit numbers. This ipynb attempts to optimize the pursuit
of a more general question of "what is the largest palindromic number that can
be created from the product of two `n` digit number?"

## Import Block (and helper functions)

In [4]:
import math, time, heapq
import numpy as np

import cProfile, io, pstats
from pstats import SortKey

In [8]:
# A function that provides the largest n digit number
#  and the smallest n digit number
def ndigit_max_min(n):
    largest = 10**n - 1
    smallest = 10**(n-1)
    return largest, smallest

# A function that checks if a provided number is a palindrome
def check_palind(x):
    def rev(num):
        ret = 0
        while num > 0:
            ret = ret*10 + num % 10
            num = num//10
        return ret
    y = rev(x)
    return x == y

# A timing function
def timing_report(test_function, no_trials, *args):
    times = np.zeros((no_trials))
    for i in range(no_trials):
        x = time.time()
        test_function(*args)
        y = time.time()
        times[i] = y-x
    print(f"Timing report: {args}")
    print(f"mean: {np.mean(times)}, std-dev: {np.std(times)}")
    print(f"span: {np.max(times)-np.min(times)}")

# profiler:
def profiling(test_function, *args):
    pr = cProfile.Profile()

    pr.enable()
    print(test_function(*args))
    pr.disable()

    s = io.StringIO()
    sortby = SortKey.CUMULATIVE
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    return ps, s

# check if int `n` has nonevel length
def nonevenLength(n):
    return (math.floor(math.log10(n)+1) % 2 == 1)



## Constant memory solutions

### The worst "reasonable" solution
Please do not loop over all possible numbers by computing the largest possible product
(`(10**n-1)**2`) and the smallest possible product (`(10**(n-1))**2`) and looping downwards,
stopping when you find the first palindrome that has two `n` digit factors. It has one massive
benefit of early stopping, but you will also loop over (and attempt to factorize) a significant
quantity of numbers that aren't factorable to two `n` digit numbers.  

However, we can learn from this solution that we should loop downwards! Especially if we can
find a way to stop early.

In [ ]:
# No, I will not be attempting to implement and optimize this potential algorithm

### A slightly better solution
Now we've learned we should loop only over the possible `n` digit pairings and that we should
loop backwards, so that we encounter larger answers first. With such knowledge in hand, we might
code something similar to something below. We can even add the subtle optimization of checking to
see if the current product we're checking is larger
than the largest palindrome we've found in order to short circuit our conditional and avoid computing
if a number is a palindrome if the number isn't even large enough to be the largest palindrome
in our search space.

In [ ]:
def palindrome(n):
    largest = 0
    big, small = ndigit_max_min(n)
    for i in range(big, small, -1):
        for j in range(big, i-1, -1):
            # subtle optimization of short circuiting
            if (i*j > largest and check_palind(i*j)):
                largest = i*j
    return largest

ps, s = profiling(palindrome, 4)
ps.print_stats()
print(s.getvalue())

### Improving the looping scheme

However, this looping scheme isn't as good as it could be. We often find ourselves checking smaller
values before we look at larger ones. For example, if `n=2`, our loop checks `97*97` before it
checks `96*99` even though the magnitude of the latter is larger than that of the former.
This means that somewhere, we can reduce the number of `n` digit pairs that we check to be 
palindromes. A better scheme for searching the `n` digit pair space is to go diagonally across
the pairing instead of straight up and down. What is kept consistent by the inner loop is the 
sum of the `n` digit pair. 

In [ ]:
def palindrome(n):
    largest = 0
    big, small = ndigit_max_min(n)
    for i in range(2*big, 2*small, -1):
        mid = i/2
        upper = math.ceil(mid)
        lower = math.floor(mid)
        while (upper <= big and lower >= small) and (upper * lower > largest):
            if check_palind(upper*lower):
                largest = upper*lower
            upper += 1
            lower -= 1
    return largest

ps, s = profiling(palindrome, 6)
ps.print_stats()
print(s.getvalue())

This provides a surprisingly ridiculous speed up, especially as `n` increases

Through all this, you might be thinking, why haven't we added an early stopping condition? That is
because both of these traversal methods fail to guarantee that our next iteration's product will
be smaller than the current iteration. Instead we must do our best to cut out work using conditionals,
and this specific traversal method gives us access to another conditional that we might even consider
early stopping.

Now that we are traversing in a manner that has a mathematical constraint of the sum being held
constant in the inner loop and decreasing in the outer loop, we can stop early if the square root
of the largest palindrome found is larger than the midpoint our inner loop is searching around. This may eventually
give diminishing returns by the nature of the square root (it will exclude a smaller proportion
of the search space as `n` increases), but is quite a speed up in the lower digit length searches,
significantly reducing the average time when looking for palindromes from multiplication by `7` digit integers.

In [13]:
def palindrome(n):
    largest = 0
    big, small = ndigit_max_min(n)
    for i in range(2*big, 2*small, -1):
        mid = i/2
        upper = math.ceil(mid)
        lower = math.floor(mid)
        while (upper <= big and lower >= small) and (upper * lower > largest):
            if check_palind(upper*lower):
                largest = upper*lower
            upper += 1
            lower -= 1
        if mid * mid <= largest:
            break
    return largest

ps, s = profiling(palindrome, 7)
ps.print_stats()
print(s.getvalue())

99956644665999
         9405196 function calls in 11.509 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.389    1.389   11.509   11.509 /var/folders/zg/t7m7vymd5vvchy5x2_w0z07r0000gn/T/ipykernel_93905/3368440410.py:1(palindrome)
  4698242    0.864    0.000   10.119    0.000 /var/folders/zg/t7m7vymd5vvchy5x2_w0z07r0000gn/T/ipykernel_93905/2611896026.py:9(check_palind)
  4698242    9.256    0.000    9.256    0.000 /var/folders/zg/t7m7vymd5vvchy5x2_w0z07r0000gn/T/ipykernel_93905/2611896026.py:10(rev)
     4336    0.001    0.000    0.001    0.000 {built-in method math.ceil}
     4336    0.000    0.000    0.000    0.000 {built-in method math.floor}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 /Users/leonardshin/opt/anaconda3/envs/EE511/lib/python3.10/site-packages/ipykernel/iostream.py:502(write)
        3    0.000    0.000    0.000 

### Using some numeric properties of palindromes.

One of the numeric properties of palindromes is that if they have an even number of digits, the
palindrome must be divisible by 11. We know that a significant quantity of our search space is
made up of numbers with an even quantity of digits. However, in our search space we are not
guaranteed that the first palindro will be found in that space. For example, when looking at `n=3`,
we know that `999*999` gives us a six digit number, but at some point, we may begin to encounter
five digit numbers, so we can reduce the search space by forcing one of the two `n` digit numbers
to be divisible by `11` when the number of digits in the product is even.

In [12]:

def palindrome(n):

    start, end = ndigit_max_min(n)
    largest = 0
    for i in range(2*start, 2*end, -1):
        mid = i/2
        upper = math.ceil(mid)
        lower = math.floor(mid)
        while (upper <= start and lower >= end) and (upper * lower > largest):
            # a palindrome with odd number of digits doesn't need to be a multiple of 10
            prod = upper*lower
            if (upper % 11 == 0 or lower % 11 == 0 or nonevenLength(prod)) and check_palind(prod):
                largest = prod
            upper += 1
            lower -= 1
        if mid * mid <= largest:
            break
    return largest

ps, s = profiling(palindrome, 7)
ps.print_stats()
with open("const_space.txt", "w") as f:
    f.write(s.getvalue())

99956644665999


## Non-constant Memory solution.
One of the worst parts of constant memory solutions is that we don't have the ability to stop
early or to know that the first solution we found is the best one. This section is dedicated to that
goal.

I'd like to thank a friend [Daniel Rui](http://danielrui.com), for seeding me the thoughts required to write this solution

### Using a priority queue and the large cost of log(n) remove and insert
This process guarantees that the first palindrome we find is the largest one, doing a bfs
over the search space. It requires a priority queue and uses it to guarantee searching from
the largest products to the smallest, so that it can return the first palindrome it finds.
However, due to the large nature of the search space and that we are attempting to add to and
pop from a priority queue that is constantly increasing in size, this solution is slower
than the memoryless solution of early stopping using the square root. 

The main reason is that once the constant space solution has found it's first palindrome, it rarely
if ever calls the function `check_palind`. It has sufficient checks on the arithmetic properties
and the magnitude of the product to prevent it from needing to do any more heavy lifting, looping
significantly more, but very easily as it does nearly nothing in most loops. However
the non-constant space solution, does almost the same thing, maybe loop a few times less, but
it does the heavy lifting of `heappop` and `heappush` in nearly every iteration. 

Despite being able to stop early, the non-constant memory solution fails to take advantage of
the speed gain from stopping at the first solution, because the constant memory solution has
practically stopped early, doing close to nothing in most of the loops after finding it's first
potential largest palindrome.


In [13]:
def palindrome(n):
    start, end = ndigit_max_min(n)
    queue = [(-1*start*start, (start, start))]
    recording = {(start, start)}
    while len(queue) > 0:
        value, pairing = heapq.heappop(queue)
        if ((value% 11 == 0 or value%11 == 0 or nonevenLength(-1*value)) and check_palind(-1*value)):
            return -1*value
        left_pair = (pairing[0]-1, pairing[1])
        if not (left_pair in recording) and left_pair[0] >= end and left_pair[1] >= end :
            heapq.heappush(queue, (-1*left_pair[0]*left_pair[1],left_pair))
            recording.add(left_pair)
        bottom_pair = (pairing[0], pairing[1]-1)
        if pairing[0] != pairing[1] and not (bottom_pair in recording) and bottom_pair[0] >= end and bottom_pair[1] >= end :
            heapq.heappush(queue, (-1*bottom_pair[0]*bottom_pair[1], bottom_pair))
            recording.add(bottom_pair)
    return None

ps, s = profiling(palindrome, 7)
ps.print_stats()
with open("non_constant.txt", "w") as f:
    f.write(s.getvalue())